## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2026-01-28-22-56-41 +0000,nypost,"Alex Pretti appears to spit at ICE, kick out t...",https://nypost.com/2026/01/28/us-news/alex-pre...
1,2026-01-28-22-54-00 +0000,wsj,Opinion | Guan Heng Gets U.S. Asylum,https://www.wsj.com/opinion/guan-heng-asylum-r...
2,2026-01-28-22-54-00 +0000,wsj,Opinion | The Perils of a Falling Trump Dollar,https://www.wsj.com/opinion/donald-trump-weak-...
3,2026-01-28-22-53-37 +0000,nyt,Hillary Clinton and Tucker Carlson Speak at Sa...,https://www.nytimes.com/2026/01/28/world/middl...
4,2026-01-28-22-53-16 +0000,nyt,Amazon’s $35 Million ‘Melania’ Promotion Has C...,https://www.nytimes.com/2026/01/28/business/me...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2487/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
24,trump,56
10,minneapolis,26
4,ice,20
1,pretti,16
158,new,16


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
49,2026-01-28-22-01-37 +0000,nypost,Bruce Springsteen blasts Trump’s ICE raids in ...,https://nypost.com/2026/01/28/us-news/bruce-sp...,142
0,2026-01-28-22-56-41 +0000,nypost,"Alex Pretti appears to spit at ICE, kick out t...",https://nypost.com/2026/01/28/us-news/alex-pre...,135
359,2026-01-28-02-00-00 +0000,wsj,Trump is looking for someone to run the Fed wh...,https://www.wsj.com/economy/central-banking/tr...,132
333,2026-01-28-04-36-00 +0000,wsj,President Trump defended Homeland Security Sec...,https://www.wsj.com/politics/policy/rump-krist...,130
252,2026-01-28-12-30-42 +0000,nypost,"Trump calls GOP Sens. Murkowski, Tillis ‘loser...",https://nypost.com/2026/01/28/us-news/gop-sens...,130


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
49,142,2026-01-28-22-01-37 +0000,nypost,Bruce Springsteen blasts Trump’s ICE raids in ...,https://nypost.com/2026/01/28/us-news/bruce-sp...
0,89,2026-01-28-22-56-41 +0000,nypost,"Alex Pretti appears to spit at ICE, kick out t...",https://nypost.com/2026/01/28/us-news/alex-pre...
359,76,2026-01-28-02-00-00 +0000,wsj,Trump is looking for someone to run the Fed wh...,https://www.wsj.com/economy/central-banking/tr...
333,74,2026-01-28-04-36-00 +0000,wsj,President Trump defended Homeland Security Sec...,https://www.wsj.com/politics/policy/rump-krist...
274,56,2026-01-28-11-10-37 +0000,nyt,Iran Killed Thousands of Protesters. Here Are ...,https://www.nytimes.com/2026/01/28/world/middl...
168,52,2026-01-28-17-30-00 +0000,wsj,The Bank of Canada kept its policy rate unchan...,https://www.wsj.com/articles/bank-of-canada-st...
327,48,2026-01-28-06-26-29 +0000,nypost,South Korea’s former first lady Kim Keon Hee s...,https://nypost.com/2026/01/28/world-news/south...
279,47,2026-01-28-11-00-00 +0000,latimes,"Immigration agents, leaders defy best practice...",https://www.latimes.com/politics/story/2026-01...
261,46,2026-01-28-12-01-00 +0000,wsj,"If American politics were a card game (hey, it...",https://www.wsj.com/politics/policy/the-two-bi...
270,45,2026-01-28-11-37-41 +0000,nypost,Trump claims Rep. Ilhan Omar ‘probably had her...,https://nypost.com/2026/01/28/us-news/trump-cl...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
